In [1]:
import geopandas as gpd
import os
import glob
import pandas as pd
pd.set_option('display.max_rows', None)

os.chdir(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\Data\RITIS")
os.getcwd()

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

print(combined_csv.shape)

df = combined_csv.drop_duplicates()

df1 = df.loc[(df['Agency-specific Sub Type'] != "Stalled Vehicle")]

df2 = df1.loc[(df1['Standardized Type'] != "Road Maintenance Operations")]
df3 = df2.loc[(df2['Standardized Type'] != "Bridge Maintenance Operations")]
trans = df3.loc[(df3['Standardized Type'] != "Utility Work")]

indexNames = trans[(trans['Standardized Type'] == 'Incident') & (trans['Agency-specific Type'] == 'Accident')].index
trans.drop(indexNames, inplace=True)

#indexNames2 = trans[(trans['Standardized Type'] == 'Congestion Due to Closure') & (trans['Agency-specific Type'] == 'Road Closed') & (trans['Agency-specific Type'] == 'Road Closed')& (trans['County'] == 'Plaquemines')].index
#trans.drop(indexNames2, inplace=True)

indexNames3 = trans[(trans['Standardized Type'] == 'Construction Work') & (trans['Agency-specific Type'] == 'Roadwork')].index
trans.drop(indexNames3, inplace=True)

trans = trans.dropna(subset=['Latitude', 'Longitude'])

(47909, 17)


C:\Users\ta444503\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [2]:
trans.shape

(35093, 17)

In [3]:
trans = trans.reset_index()

In [4]:
trans['Agency-specific Type'].unique()

array(['Weatherhazard', 'Road Closed', 'Accident', 'Roadwork',
       'Weatherclosures', 'Closures', 'Accidentsandincidents',
       'Weatherconditions', 'Incident', 'Obstruction', 'Specialevents',
       nan], dtype=object)

In [4]:
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon

In [5]:
cousb = gpd.read_file(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\tl_2016_22_cousub\tl_2016_22_cousub.shp")

In [6]:
pts = []
for i,j in zip(trans['Longitude'].tolist(), trans['Latitude'].tolist()):
    pts.append(Point(i,j))

In [7]:
print(len(pts))
trans['point'] = pts

35093


C:\Users\ta444503\AppData\Roaming\Python\Python39\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [8]:
trans = trans[trans["County"] != "Hancock"]

In [9]:
trans.shape

(35090, 19)

In [10]:
trans['Longitude'] = trans['Longitude'] .astype(str)
trans['Latitude'] = trans['Latitude'] .astype(str)

In [11]:
ind = trans[(trans["Longitude"] == '-93.701384') & (trans["Latitude"] == '30.127388')].index
trans.drop(ind, inplace=True)

In [12]:
trans.shape

(35085, 19)

In [13]:
ind

Int64Index([25585, 25599, 34553, 34765, 34848], dtype='int64')

In [14]:
trans = trans.reset_index()

In [15]:
bools = []

for i in trans['point']:
    for j in cousb['geometry']:
        bools.append(j.intersects(i))

In [16]:
len(cousb)

579

In [17]:
mult_list = list(map(list, zip(*([iter(bools)]*579))))
print(len(mult_list))

35085


In [18]:
pos = []
for j in mult_list:
    pos.append([i for i, x in enumerate(j) if x])
    
print(len(pos))

35085


In [19]:
trans['pos'] = pos

In [20]:
#trans['pos'] = trans['pos'].replace('[','').replace(']','')  
trans['pos']=trans['pos'].map(lambda x: str(x).lstrip('[]').rstrip(']'))

In [21]:
trans.head()

,level_0,index,Agency,Standardized Type,Agency-specific Type,Agency-specific Sub Type,Start time,Closed time,Location,Op. Center,...,Longitude,Road,Direction,County,Roadway Clearance Time,Operator Notes,Responders,Max Lanes Closed,point,pos
0,0,0,Waze,Obstructions,Weatherhazard,Hazard On Road Object,2021-09-12T22:48:33+00:00,2021-09-13T00:08:11+00:00,US-90,Unknown,...,-91.773374,US 90,West,Iberia,1 hour 19 minutes,NaN,NaN,NaN,POINT (-91.773374 29.942577),3
1,1,2,Waze,Disabled Vehicle,Weatherhazard,Hazard On Road Car Stopped,2021-09-12T20:16:29+00:00,2021-09-12T20:45:26+00:00,I-10,Unknown,...,-90.713972,I-10,East,St. James,28 minutes,NaN,NaN,NaN,POINT (-90.713972 30.129774),452
2,2,4,Waze,Obstructions,Weatherhazard,Hazard On Road Object,2021-09-12T19:16:55+00:00,2021-09-12T19:50:08+00:00,PERKINS RD,Unknown,...,-91.117272,PERKINS RD,NaN,East Baton Rouge,33 minutes,NaN,NaN,NaN,POINT (-91.117272 30.40024),164
3,3,5,Waze,Incident,Weatherhazard,Hazard On Shoulder Missing Sign,2021-09-12T13:47:10+00:00,2021-09-12T14:22:11+00:00,I-10,Unknown,...,-90.660647,I-10,West,St. James,35 minutes,NaN,NaN,NaN,POINT (-90.660647 30.121607),452
4,4,6,Waze,Construction Work,Weatherhazard,Hazard On Road Construction,2021-09-12T20:36:44+00:00,2021-09-12T21:20:07+00:00,LA-25,Unknown,...,-90.123756,LA 25,NaN,St. Tammany,43 minutes,NaN,NaN,NaN,POINT (-90.123756 30.52299),139


In [22]:
trans['Agency-specific Type'].unique()

array(['Weatherhazard', 'Road Closed', 'Accident', 'Roadwork',
       'Weatherclosures', 'Closures', 'Accidentsandincidents',
       'Weatherconditions', 'Incident', 'Obstruction', 'Specialevents',
       nan], dtype=object)

In [161]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35085 entries, 0 to 35084
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   level_0                   35085 non-null  int64  
 1   index                     35085 non-null  int64  
 2   Agency                    35085 non-null  object 
 3   Standardized Type         35085 non-null  object 
 4   Agency-specific Type      35050 non-null  object 
 5   Agency-specific Sub Type  34540 non-null  object 
 6   Start time                35085 non-null  object 
 7   Closed time               35079 non-null  object 
 8   Location                  34971 non-null  object 
 9   Op. Center                35085 non-null  object 
 10  Latitude                  35085 non-null  object 
 11  Longitude                 35085 non-null  object 
 12  Road                      34164 non-null  object 
 13  Direction                 26144 non-null  object 
 14  County

In [164]:
#my_str = "Hello from AskPython"
check = []
target = ","
for i in trans['pos']: 
    if (i.__contains__(target)):
        check.append(i)
        #trans = t
        #print("String contains target!")
    #else:
        #print("String does not contain target")

In [165]:
check

['501, 563',
 '22, 24',
 '157, 161',
 '225, 526',
 '158, 248',
 '124, 127',
 '38, 369']

In [166]:
trans = trans[trans['pos'] != '501, 563']
trans = trans[trans['pos'] != '22, 24']
trans = trans[trans['pos'] != '157, 161']
trans = trans[trans['pos'] != '225, 526']
trans = trans[trans['pos'] != '158, 248']
trans = trans[trans['pos'] != '124, 127']
trans = trans[trans['pos'] != '38, 369']

In [167]:
print(len(trans))

35078


In [168]:
trans['pos'] = pd.to_numeric(trans['pos'])

In [169]:
county_subdivision = []

for i in trans['pos']:
    county_subdivision.append(cousb["NAMELSAD"].iloc[i])

In [171]:
len(county_subdivision)

35078

In [172]:
trans["County Subdivision"] = county_subdivision

In [173]:
trans = trans.drop(columns = ['level_0','index','Operator Notes','Responders','Max Lanes Closed','Direction'])

In [174]:
trans.head()

,Agency,Standardized Type,Agency-specific Type,Agency-specific Sub Type,Start time,Closed time,Location,Op. Center,Latitude,Longitude,Road,County,Roadway Clearance Time,point,pos,County Subdivision
0,Waze,Obstructions,Weatherhazard,Hazard On Road Object,2021-09-12T22:48:33+00:00,2021-09-13T00:08:11+00:00,US-90,Unknown,29.942577,-91.773374,US 90,Iberia,1 hour 19 minutes,POINT (-91.773374 29.942577),3,District 10
1,Waze,Disabled Vehicle,Weatherhazard,Hazard On Road Car Stopped,2021-09-12T20:16:29+00:00,2021-09-12T20:45:26+00:00,I-10,Unknown,30.129774,-90.713972,I-10,St. James,28 minutes,POINT (-90.713972 30.129774),452,District 1
2,Waze,Obstructions,Weatherhazard,Hazard On Road Object,2021-09-12T19:16:55+00:00,2021-09-12T19:50:08+00:00,PERKINS RD,Unknown,30.40024,-91.117272,PERKINS RD,East Baton Rouge,33 minutes,POINT (-91.117272 30.40024),164,District 12
3,Waze,Incident,Weatherhazard,Hazard On Shoulder Missing Sign,2021-09-12T13:47:10+00:00,2021-09-12T14:22:11+00:00,I-10,Unknown,30.121607,-90.660647,I-10,St. James,35 minutes,POINT (-90.660647 30.121607),452,District 1
4,Waze,Construction Work,Weatherhazard,Hazard On Road Construction,2021-09-12T20:36:44+00:00,2021-09-12T21:20:07+00:00,LA-25,Unknown,30.52299,-90.123756,LA 25,St. Tammany,43 minutes,POINT (-90.123756 30.52299),139,District 2


In [175]:
trans['Start time'] = pd.to_datetime(trans['Start time'], utc = True)
trans['Date'] = trans['Start time'].dt.date   # date means start date
trans['Closed time'] = pd.to_datetime(trans['Closed time'], utc = True)
trans['End Date'] = trans['Closed time'].dt.date

trans['obstruction_time'] = (trans['Closed time'] - trans['Start time']) / pd.Timedelta(hours=1)

In [176]:
trans['County'] = trans['County'].str.replace(' Parish', '')
trans['Place'] = trans['County Subdivision'] + ', ' + trans['County']

In [177]:
len(trans['Place'].unique().tolist())

453

In [178]:
trans['Date'] = pd.to_datetime(trans['Date'])
limited_time2 = trans.loc[(trans['Date'] >= "2021-08-25")]
duration_road_blockage3 = limited_time2.groupby("Place")["obstruction_time"].sum()

In [179]:
duration_road_blockage3 

Place
District 1, Acadia                      0.553056
District 1, Ascension                   1.647500
District 1, Assumption                  3.185000
District 1, Bienville                  13.486111
District 1, Bossier                    17.847500
District 1, Caddo                      80.673889
District 1, Calcasieu                   3.365833
District 1, Caldwell                    1.098889
District 1, Cameron                   113.185000
District 1, Catahoula                   0.523889
District 1, East Baton Rouge          695.379722
District 1, East Carroll               23.427222
District 1, Evangeline               1091.711111
District 1, Iberia                      4.297500
District 1, Iberville                   0.557778
District 1, Jackson                     3.478889
District 1, Jefferson                3439.519722
District 1, La Salle                    0.553333
District 1, Lafourche                   4.810278
District 1, Lincoln                    23.150556
District 1, Li

# after_19thSept

In [201]:
after_19thSept = pd.read_csv(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\Data\extra ritis\ritis-events 20 sept to 30 sept.csv")

In [202]:
after_19thSept2 = after_19thSept.drop_duplicates()

In [203]:
df11 = after_19thSept2.loc[(after_19thSept2['Agency-specific Sub Type'] != "Stalled Vehicle")]

df12 = df11.loc[(df11['Standardized Type'] != "Road Maintenance Operations")]
df13 = df12.loc[(df12['Standardized Type'] != "Bridge Maintenance Operations")]
trans11 = df13.loc[(df13['Standardized Type'] != "Utility Work")]

In [204]:
indexNames11 = trans11[(trans11['Standardized Type'] == 'Incident') & (trans11['Agency-specific Type'] == 'Accident')].index
trans11.drop(indexNames11, inplace=True)

#indexNames12 = trans11[(trans11['Standardized Type'] == 'Congestion Due to Closure') & (trans11['Agency-specific Type'] == 'Road Closed') & (trans11['Agency-specific Type'] == 'Road Closed')& (trans11['County'] == 'Plaquemines')].index
#trans11.drop(indexNames12, inplace=True)

indexNames13 = trans11[(trans11['Standardized Type'] == 'Construction Work') & (trans11['Agency-specific Type'] == 'Roadwork')].index
trans11.drop(indexNames13, inplace=True)

trans11 = trans11.dropna(subset=['Latitude', 'Longitude'])

C:\Users\ta444503\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [205]:
trans11 = trans11.reset_index()

In [206]:
pts2 = []
for i,j in zip(trans11['Longitude'].tolist(), trans11['Latitude'].tolist()):
    pts2.append(Point(i,j))

len(pts2)

18800

In [207]:
trans11['point'] = pts2
trans11 = trans11[trans11["County"] != "Hancock"]

C:\Users\ta444503\AppData\Roaming\Python\Python39\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [208]:
bools2 = []

for i in trans11['point']:
    for j in cousb['geometry']:
        bools2.append(j.intersects(i))

In [209]:
mult_list2 = list(map(list, zip(*([iter(bools2)]*579))))

In [210]:
pos2 = []
for j in mult_list2:
    pos2.append([i for i, x in enumerate(j) if x])

In [211]:
trans11['pos'] = pos2

In [214]:
#trans11 = trans11[trans11['pos'] != "[]"]

In [216]:
#trans11.to_csv(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\b.csv",index=False)

In [217]:
trans11['pos']=trans11['pos'].map(lambda x: str(x).lstrip('[]').rstrip(']'))

In [218]:
trans11.head()

,index,Agency,Standardized Type,Agency-specific Type,Agency-specific Sub Type,Start time,Closed time,Location,Op. Center,Latitude,...,Road,Direction,County,Roadway Clearance Time,Duration (Incident clearance time),Operator Notes,Responders,Max Lanes Closed,point,pos
0,0,Waze,Construction Work,Weatherhazard,Hazard On Road Construction,2021-09-20T21:13:29+00:00,2021-09-20T22:41:08+00:00,US-90,Unknown,29.910148,...,US 90,West,St. Charles,1 hour 27 minutes,1 hour 27 minutes,NaN,NaN,NaN,POINT (-90.35803 29.910148),384
1,1,Waze,Construction Work,Weatherhazard,Hazard On Road Construction,2021-09-21T02:44:08+00:00,2021-09-21T04:15:46+00:00,I-12,Unknown,30.474068,...,I-12,East,Livingston,1 hour 31 minutes,1 hour 31 minutes,NaN,NaN,NaN,POINT (-90.756192 30.474068),519
2,2,Waze,Incident,Weatherhazard,Hazard On Shoulder Missing Sign,2021-09-20T22:14:10+00:00,2021-09-20T22:25:58+00:00,I-10,Unknown,30.246327,...,I-10,East,Jefferson Davis,11 minutes,11 minutes,NaN,NaN,NaN,POINT (-92.64672899999999 30.246327),470
3,3,Waze,Collision,Accident,Accident Minor,2021-09-20T20:46:26+00:00,2021-09-20T21:07:22+00:00,US-90,Unknown,29.945499,...,US 90,West,Orleans,20 minutes,20 minutes,NaN,NaN,NaN,POINT (-90.079391 29.945499),572
4,4,Waze,Obstructions,Weatherhazard,Hazard On Road Object,2021-09-20T20:47:11+00:00,2021-09-20T21:08:12+00:00,US-90,Unknown,29.738992,...,US 90,East,Lafourche,21 minutes,21 minutes,NaN,NaN,NaN,POINT (-90.557227 29.738992),233


In [219]:
trans11.shape

(18799, 21)

In [220]:
len(pos2)

18799

In [198]:
#trans11 = trans11['pos'].dropna()

In [199]:
trans11.shape

(18799,)

In [221]:
#my_str = "Hello from AskPython"
check = []
target = ","
for i in trans11['pos']: 
    if (i.__contains__(target)):
        check.append(i)
        #trans = t
        #print("String contains target!")
    #else:
        #print("String does not contain target")

In [222]:
check

['418, 419',
 '418, 419',
 '418, 419',
 '124, 127',
 '251, 256',
 '161, 417',
 '251, 256',
 '418, 419',
 '418, 419',
 '145, 511',
 '418, 419',
 '418, 419',
 '418, 419']

In [223]:
trans11 = trans11[trans11['pos'] != '418, 419']
trans11 = trans11[trans11['pos'] != '124, 127']
trans11 = trans11[trans11['pos'] != '251, 256']
trans11 = trans11[trans11['pos'] != '161, 417']
trans11 = trans11[trans11['pos'] != '145, 511']

In [226]:
trans11.shape

(18785, 21)

In [225]:
trans11 = trans11[trans11['pos'] != '']

In [227]:
trans11['pos'] = pd.to_numeric(trans11['pos'])

In [228]:
county_subdivision2 = []

for i in trans11['pos']:
    county_subdivision2.append(cousb["NAMELSAD"].iloc[i])

In [229]:
trans11["County Subdivision"] = county_subdivision2

In [231]:
trans11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18785 entries, 0 to 18799
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   index                               18785 non-null  int64  
 1   Agency                              18785 non-null  object 
 2   Standardized Type                   18785 non-null  object 
 3   Agency-specific Type                18650 non-null  object 
 4   Agency-specific Sub Type            16155 non-null  object 
 5   Start time                          18785 non-null  object 
 6   Closed time                         18773 non-null  object 
 7   Location                            18700 non-null  object 
 8   Op. Center                          18785 non-null  object 
 9   Latitude                            18785 non-null  float64
 10  Longitude                           18785 non-null  float64
 11  Road                                18000

In [233]:
trans11 = trans11.drop(columns = ['index','Operator Notes','Responders','Max Lanes Closed','Direction','Duration (Incident clearance time)'])

In [237]:
trans11['Start time'] = pd.to_datetime(trans11['Start time'], utc = True)
trans11['Date'] = trans11['Start time'].dt.date   # date means start date
trans11['Closed time'] = pd.to_datetime(trans11['Closed time'], utc = True)
trans11['End Date'] = trans11['Closed time'].dt.date

trans11['obstruction_time'] = (trans11['Closed time'] - trans11['Start time']) / pd.Timedelta(hours=1)

trans11['County'] = trans11['County'].str.replace(' Parish', '')
trans11['Place'] = trans11['County Subdivision'] + ', ' + trans11['County']

In [238]:
trans11['Date'] = pd.to_datetime(trans11['Date'])
limited_time3 = trans11.loc[(trans11['Date'] >= "2021-08-25")]
duration_road_blockage4 = limited_time3.groupby("Place")["obstruction_time"].sum()

In [239]:
duration_road_blockage4

Place
District 1, Acadia                      1.060556
District 1, Ascension                  41.631944
District 1, Assumption                  1.581389
District 1, Beauregard                  1.193333
District 1, Bienville                   6.111111
District 1, Bossier                     9.045556
District 1, Caddo                      38.667778
District 1, Calcasieu                   6.561667
District 1, Cameron                    18.006389
District 1, Catahoula                   1.557222
District 1, Concordia                   2.171111
District 1, East Baton Rouge           41.981944
District 1, Evangeline                  0.537222
District 1, Iberia                      0.659444
District 1, Jackson                     0.526944
District 1, Jefferson                 157.597500
District 1, Jefferson Davis             0.531944
District 1, La Salle                    0.560000
District 1, Lafourche                   1.452500
District 1, Lincoln                     6.900000
District 1, Li

In [241]:
frames = [trans, trans11]

result = pd.concat(frames)

In [242]:
result = result.dropna(subset=['obstruction_time'])

In [243]:
duration_road_blockage2 = result.groupby("Place")["obstruction_time"].sum()
duration_road_blockage2 = duration_road_blockage2.reset_index()

In [245]:
result.loc[result['Closed time'] > '2021-09-30', 'Closed time'] = '2021-09-30'

In [246]:
result['obstruction_time2'] = (result['Closed time'] - result['Start time']) / pd.Timedelta(hours=1)

In [247]:
duration_road_blockage4= result.groupby("Place")["obstruction_time2"].sum()
duration_road_blockage4 = duration_road_blockage4.reset_index()

In [248]:
duration_road_blockage4

,Place,obstruction_time2
0,"District 1, Acadia",1.613611
1,"District 1, Ascension",45.357222
2,"District 1, Assumption",-5.174722
3,"District 1, Beauregard",1.193333
4,"District 1, Bienville",23.063333
5,"District 1, Bossier",27.060000
6,"District 1, Caddo",125.885278
7,"District 1, Calcasieu",10.962778
8,"District 1, Caldwell",1.710278
9,"District 1, Cameron",131.191389


In [249]:
resilience = pd.read_csv(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\resilience_166countysubdivision.csv")

In [250]:
resilience['County_subdivision'] = resilience['County_subdivision'].str.replace(' Parish', '')

In [251]:
indexNames100 = result[(result['Standardized Type'] == 'Congestion Due to Closure') & (result['Agency-specific Type'] == 'Road Closed') & (result['Agency-specific Type'] == 'Road Closed')& (result['County'] == 'East Baton Rouge')].index
result.drop(indexNames100, inplace=True)

In [253]:
time_limit = result.loc[(result['Date'] >= "2021-08-25")]

In [273]:
time_limit.to_csv(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\Data\RITIS\outut from ritis data\processed_ritis_with_all_plaquemines.csv",index=False)

In [254]:
duration_road_blockage5= time_limit.groupby("Place")["obstruction_time2"].sum()
duration_road_blockage5 = duration_road_blockage5.reset_index()

In [255]:
duration_road_blockage5

,Place,obstruction_time2
0,"District 1, Acadia",1.613611
1,"District 1, Ascension",43.279444
2,"District 1, Assumption",-5.174722
3,"District 1, Beauregard",1.193333
4,"District 1, Bienville",19.597222
5,"District 1, Bossier",26.893056
6,"District 1, Caddo",119.341667
7,"District 1, Calcasieu",9.927500
8,"District 1, Caldwell",1.098889
9,"District 1, Cameron",131.191389


In [256]:
resilience=resilience.rename(columns={'County_subdivision' : 'Place'})

In [259]:
duration_road_blockage5['Place'] = duration_road_blockage5['Place'].str.replace(' city', '')

In [260]:
tri = resilience.merge(duration_road_blockage5,how='outer')

In [261]:
tri = tri.dropna(subset=['Resilience'])


In [270]:
tri.loc[tri.Place == 'District 8, Plaquemines', 'obstruction_time2'] = 300


In [263]:
tri = tri.fillna(0.5)
tri.loc[tri.Place == 'District 3, Cameron', 'obstruction_time2'] = 7.5

In [264]:
tri['Abs_Loss_of_Resilience'] = tri['Loss_of_Resilience']*(-1)

In [271]:
column_5 = tri['Abs_Loss_of_Resilience']
column_6 = tri['obstruction_time2']
correlation3 = column_5.corr(column_6)
print(correlation3)

0.2862160566728362


In [268]:
tri1 = tri[tri['Place']!='District 8, Plaquemines']

In [269]:
column_5 = tri1['Abs_Loss_of_Resilience']
column_6 = tri1['obstruction_time2']
correlation3 = column_5.corr(column_6)
print(correlation3)

0.30062014539504567


In [272]:
tri

,Place,Loss_of_Resilience,Resilience,obstruction_time2,Abs_Loss_of_Resilience
0,"District 4B, East Feliciana",-1.049961,34.950039,57.263333,1.049961
1,"District 3, St. John the Baptist",-4.727669,31.272331,41.376111,4.727669
2,"District 3, Washington",-0.394778,35.605222,1.851667,0.394778
3,"District 3, St. Martin",-2.390723,33.609277,1.583611,2.390723
4,"District 2, St. Tammany",-1.602185,34.397815,198.328611,1.602185
5,"District 2, Lafourche",-2.331524,33.668476,87.294722,2.331524
6,"District 2, Jefferson",-5.247344,30.752656,2803.500833,5.247344
7,"District 8, Ascension",-0.294148,35.705852,118.960278,0.294148
8,"District 6, Terrebonne",-2.150317,33.849683,70.199167,2.150317
9,"District 6, Plaquemines",-9.606036,26.393964,11441.321667,9.606036


In [274]:
tri.to_csv(r"C:\Users\ta444503\OneDrive - Knights - University of Central Florida\Desktop\Spring 2022\Algorithm\ida\Data\RITIS\outut from ritis data\duration_roadblockage_with_all_plaquemines.csv",index=False)